# Auction Reclean

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
import seaborn as sns

In [2]:
bid_ytd=pd.read_csv('all_bid.csv')
bid_ytd.head()

/Users/sydneybednar/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,auction_id,user_id,buyer,item_id,brand_name,category_name,title,overall_grade,bid,min_bid,winning_price,bid_placed_on,hour,day,date,auction_name,buyer_id,win,within_10
0,250,182137.0,karen's rocks and rags,2131275,Louis Vuitton,Clutches,Louis Vuitton Trousse Toilette,C,99,9.84,137.0,1/1/21 12:11,12,4,1/1/21,Aucnet Low,NaN,N,N
1,250,182137.0,karen's rocks and rags,2138374,Christian Dior,Clutches,Christian Dior Bag / pouch pink,C,49,14.77,47.0,1/1/21 12:13,12,4,1/1/21,Aucnet Low,NaN,Y,Y
2,250,182137.0,karen's rocks and rags,2135793,Chanel,Clutches,Chanel Patent bag pouch black,C,79,14.77,250.0,1/1/21 12:14,12,4,1/1/21,Aucnet Low,NaN,N,N
3,250,182137.0,karen's rocks and rags,2131983,Louis Vuitton,Clutches,Louis Vuitton Etui Troyes Balldu Golf Pouch,C,47,14.77,NaN,1/1/21 12:15,12,4,1/1/21,Aucnet Low,NaN,N,N
4,250,182137.0,karen's rocks and rags,2135036,Christian Dior,Bracelets,Christian Dior Bangle Bracelet metal,B,99,14.77,NaN,1/1/21 12:16,12,4,1/1/21,Aucnet Low,NaN,N,N


In [3]:
bid_ytd['buyer']=bid_ytd['buyer'].str.lower()

In [4]:
bid_ytd['bid_placed_on']=pd.to_datetime(bid_ytd['bid_placed_on'])
bid_ytd['day']=bid_ytd['bid_placed_on'].dt.dayofweek
bid_ytd['date']=bid_ytd['bid_placed_on'].dt.date
bid_ytd['hour']=bid_ytd['bid_placed_on'].dt.hour

In [5]:
bid_ytd.head()

,auction_id,user_id,buyer,item_id,brand_name,category_name,title,overall_grade,bid,min_bid,winning_price,bid_placed_on,hour,day,date,auction_name,buyer_id,win,within_10
0,250,182137.0,karen's rocks and rags,2131275,Louis Vuitton,Clutches,Louis Vuitton Trousse Toilette,C,99,9.84,137.0,2021-01-01 12:11:00,12,4,2021-01-01,Aucnet Low,NaN,N,N
1,250,182137.0,karen's rocks and rags,2138374,Christian Dior,Clutches,Christian Dior Bag / pouch pink,C,49,14.77,47.0,2021-01-01 12:13:00,12,4,2021-01-01,Aucnet Low,NaN,Y,Y
2,250,182137.0,karen's rocks and rags,2135793,Chanel,Clutches,Chanel Patent bag pouch black,C,79,14.77,250.0,2021-01-01 12:14:00,12,4,2021-01-01,Aucnet Low,NaN,N,N
3,250,182137.0,karen's rocks and rags,2131983,Louis Vuitton,Clutches,Louis Vuitton Etui Troyes Balldu Golf Pouch,C,47,14.77,NaN,2021-01-01 12:15:00,12,4,2021-01-01,Aucnet Low,NaN,N,N
4,250,182137.0,karen's rocks and rags,2135036,Christian Dior,Bracelets,Christian Dior Bangle Bracelet metal,B,99,14.77,NaN,2021-01-01 12:16:00,12,4,2021-01-01,Aucnet Low,NaN,N,N


In [6]:
auc_key=pd.read_excel('AuctionSellers.xlsx')

In [7]:
auc_key

,auction_id,username
0,1,aucnet
1,2,aucnet
2,3,aucnet
3,4,aucnet
4,5,aucnet
...,...,...
325,319,camille+ecoring
326,322,camille+ecoring
327,329,camille+ecoring
328,330,camille+ecoring


In [8]:
bid_ytd.groupby('auction_id').count()

,user_id,buyer,item_id,brand_name,category_name,title,overall_grade,bid,min_bid,winning_price,bid_placed_on,hour,day,date,auction_name,buyer_id,win,within_10
auction_id,,,,,,,,,,,,,,,,,,
157,0,101,101,101,101,101,101,101,101,34,101,101,101,101,0,101,101,101
158,0,645,645,645,645,645,645,645,645,538,645,645,645,645,0,645,645,645
159,0,27,27,27,27,27,27,27,27,25,27,27,27,27,0,27,27,27
160,0,97,97,97,97,97,97,97,97,59,97,97,97,97,0,97,97,97
161,0,551,551,551,551,551,551,551,551,503,551,551,551,551,0,551,551,551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,0,394,394,394,394,394,394,394,394,389,394,394,394,394,394,0,394,394
329,0,1357,1357,1357,1357,1357,1357,1357,1357,1079,1357,1357,1357,1357,1357,0,1357,1357
330,0,605,605,605,605,605,605,605,605,0,605,605,605,605,605,0,605,605


In [9]:
def sort_auction(df):
    """
    Returns a dict. Sorts auctions into one of four categories: Aucnet High, Aucnet Low, 
    Val Bags & Jewelry, and EcoRing.
    
    Parameter df: the DataFrame to search
    Precondition: df is a pandas DataFrame
    """
    dict={}
    
    grp=df.groupby('auction_id')
    day_df=pd.DataFrame(grp['day'].last())
    time_df=pd.DataFrame(grp['hour'].first())
    
    for i in grp.groups:
        end_day=day_df.loc[day_df.index==i]['day'][i]
        
        if end_day==6:
            dict[i]='Aucnet Low'
        elif end_day==0:
            start_hour=time_df.loc[time_df.index==i]['hour'][i]
            if list(auc_key.loc[auc_key['auction_id']==i]['username'])[0]=='camille+ecoring':
                dict[i]='EcoRing'
            else:
                dict[i]='Aucnet High'
                
        elif end_day==1:
            dict[i]='Val Bags & Jewelry'          
            
    return dict
aucs=sort_auction(bid_ytd)

In [14]:
new_auc_key=pd.DataFrame.from_dict(aucs, orient='index')

bid_ytd=pd.merge(bid_ytd, new_auc_key, left_on='auction_id', right_index=True, how='left')

In [12]:
bid_ytd.head()

,auction_id,user_id,buyer,item_id,brand_name,category_name,title,overall_grade,bid,min_bid,winning_price,bid_placed_on,hour,day,date,auction_name_x,buyer_id,win,within_10,auction_name_y
0,250,182137.0,karen's rocks and rags,2131275,Louis Vuitton,Clutches,Louis Vuitton Trousse Toilette,C,99,9.84,137.0,2021-01-01 12:11:00,12,4,2021-01-01,Aucnet Low,NaN,N,N,NaN
1,250,182137.0,karen's rocks and rags,2138374,Christian Dior,Clutches,Christian Dior Bag / pouch pink,C,49,14.77,47.0,2021-01-01 12:13:00,12,4,2021-01-01,Aucnet Low,NaN,Y,Y,NaN
2,250,182137.0,karen's rocks and rags,2135793,Chanel,Clutches,Chanel Patent bag pouch black,C,79,14.77,250.0,2021-01-01 12:14:00,12,4,2021-01-01,Aucnet Low,NaN,N,N,NaN
3,250,182137.0,karen's rocks and rags,2131983,Louis Vuitton,Clutches,Louis Vuitton Etui Troyes Balldu Golf Pouch,C,47,14.77,NaN,2021-01-01 12:15:00,12,4,2021-01-01,Aucnet Low,NaN,N,N,NaN
4,250,182137.0,karen's rocks and rags,2135036,Christian Dior,Bracelets,Christian Dior Bangle Bracelet metal,B,99,14.77,NaN,2021-01-01 12:16:00,12,4,2021-01-01,Aucnet Low,NaN,N,N,NaN


In [13]:
# bid_ytd.to_csv('all_bid.csv')